In [0]:
%scala

val storageAccount = "hsotomontalvo1"  //nombre de la cuenta de almacenamiento
val container = "hsotomontalvocont"  //nombre del contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-04T13:23:52Z&st=2024-12-04T05:23:52Z&spr=https&sig=BgMMBbTluy5WJv%2BgKW2xnxoSwP5NTyyhbpn5qlpdsPA%3D"   //llave conexion

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"


//Crear acceso directo
dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = hsotomontalvo1
container: String = hsotomontalvocont
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-04T13:23:52Z&st=2024-12-04T05:23:52Z&spr=https&sig=BgMMBbTluy5WJv%2BgKW2xnxoSwP5NTyyhbpn5qlpdsPA%3D
conf: String = fs.azure.sas.hsotomontalvocont.hsotomontalvo1.blob.core.windows.net
res15: Boolean = true

In [0]:
##Ingestamos archivo Producto.parquet

producto = (spark.read.parquet("/mnt/hsotomontalvocont/proyectofinal/capaplata/Product.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
##Ingestamos archivo Categoria.parquet
categoria = (spark.read.parquet("/mnt/hsotomontalvocont/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
##Ingestamos archivo SubCategoria.parquet
subcategoria = (spark.read.parquet("/mnt/hsotomontalvocont/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql

---Para eliminar la tabla de la base de datos
DROP TABLE IF EXISTS Fact_Product;

--crear tabla
CREATE TEMP VIEW Fact_Product
AS 
SELECT P.Cod_Producto, P.Nombre As Producto, S.Nombre As SubCategoria, C.Nombre As Categoria
FROM s_producto P
Join s_subcategoria S on P.Cod_SubCategoria = S.Cod_SubCategoria
JOIN s_categoria C ON S.Cod_Categoria = C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/hsotomontalvocont/proyectofinal/capaoro/FactProduct.parquet")

In [0]:
#Eliminar acceso directo
dbutils.fs.unmount("/mnt/hsotomontalvocont")

/mnt/hsotomontalvocont has been unmounted.


True